# Con que páginas vamos a trabajar
• El Mundo (https://elmundo.es) 
- Salud: https://www.elmundo.es/ciencia-y-salud/salud.html 
- Tecnología: https://www.elmundo.es/tecnologia.html 
- Ciencia: https://www.elmundo.es/ciencia-y-salud/ciencia.html 
 
• El País (https://elpais.com/) 
- Sanidad: https://elpais.com/noticias/sanidad/ 
- Tecnología: https://elpais.com/tecnologia/ 
- Ciencia: https://elpais.com/ciencia/ 

• 20 Minutos (https://www.20minutos.es/) 
- Salud: https://www.20minutos.es/salud/ 
- Tecnología: https://www.20minutos.es/tecnologia/ 
- Ciencia: https://www.20minutos.es/ciencia/ 

# Importamos las librerias que vamos a usar

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd
import re
import os
import PySimpleGUI as sg

# Librerías para calcular la similitud del coseno y el TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1.) Comenzamos accediendo a las URLs y sacando los links de las noticias de cada campo

In [2]:
ElMundoSalud = requests.get("https://www.elmundo.es/ciencia-y-salud/salud.html")
ElMundoTecnologia = requests.get("https://www.elmundo.es/tecnologia.html")
ElMundoCiencia = requests.get("https://www.elmundo.es/ciencia-y-salud/ciencia.html")

ElPaisSanidad = requests.get("https://elpais.com/noticias/sanidad/")
ElPaisTecnologia = requests.get("https://elpais.com/tecnologia/")
ElPaisCiencia = requests.get("https://elpais.com/ciencia/")

MinutosSalud = requests.get("https://www.20minutos.es/salud/")
MinutosTecnologia = requests.get("https://www.20minutos.es/tecnologia/")
MinutosCiencia = requests.get("https://www.20minutos.es/ciencia/")

In [3]:
# vamos a ver si ha accedido correctamente a todas las paginas (200)
print(ElMundoSalud.status_code)
print(ElMundoTecnologia.status_code)
print(ElMundoCiencia.status_code)

print(ElPaisSanidad.status_code)
print(ElPaisTecnologia.status_code)
print(ElPaisCiencia.status_code)

print(MinutosSalud.status_code)
print(MinutosTecnologia.status_code)
print(MinutosCiencia.status_code)

200
200
200
200
200
200
200
200
200


In [4]:
# descargamos el contenido de las páginas
ContenidoElMundoSalud = BeautifulSoup (ElMundoSalud.content, 'html.parser')
ContenidoElMundoTecnologia = BeautifulSoup (ElMundoTecnologia.content, 'html.parser')
ContenidoElMundoCiencia = BeautifulSoup (ElMundoCiencia.content, 'html.parser')

ContenidoElPaisSanidad = BeautifulSoup (ElPaisSanidad.content, 'html.parser')
ContenidoElPaisTecnologia = BeautifulSoup (ElPaisTecnologia.content, 'html.parser')
ContenidoElPaisCiencia = BeautifulSoup (ElPaisCiencia.content, 'html.parser')

ContenidoMinutosSalud = BeautifulSoup (MinutosSalud.content, 'html.parser')
ContenidoMinutosTecnologia = BeautifulSoup (MinutosTecnologia.content, 'html.parser')
ContenidoMinutosCiencia = BeautifulSoup (MinutosCiencia.content, 'html.parser')

## Noticias El Mundo

In [5]:
def obtenLinksMundo(contenido, array):
    a = contenido.find_all('a', class_="ue-c-cover-content__link")
    for link in a:
        array.append(link["href"])
    return len(array)

#### Salud

In [6]:
linksSalud = []
obtenLinksMundo(ContenidoElMundoSalud, linksSalud)

50

#### Tecnologia

In [7]:
linksTecnologia = []
obtenLinksMundo(ContenidoElMundoTecnologia, linksTecnologia)

58

#### Ciencia

In [8]:
linksCiencia = []
obtenLinksMundo(ContenidoElMundoCiencia, linksCiencia)

50

## Noticias El País

In [9]:
def obtenLinksPais(contenido, array):
    h2 = contenido.find_all('h2', class_="c_t")
    for a in h2:
        for link in a:
            array.append("https://elpais.com/"+link["href"])
    return len(array)

#### Sanidad

In [10]:
linksSanidadElPáis = []
obtenLinksPais(ContenidoElPaisSanidad, linksSanidadElPáis)

27

#### Tecnologia

In [11]:
linksTecnologiaElPáis = []
obtenLinksPais(ContenidoElPaisTecnologia, linksTecnologiaElPáis)

30

#### Ciencia

In [12]:
linksCienciaElPáis = []
obtenLinksPais(ContenidoElPaisCiencia, linksCienciaElPáis)

40

## Noticias 20 minutos

In [13]:
# Metodo para recuperar todos los links de "20 Minutos"

def obtenLinksMinutos(Contenido, array):
    div = Contenido.find_all('div', class_='media-content')
    for header in div:
        for a in header.findAll("a"):
            array.append(a["href"])
    return len(array)

#### Salud

In [14]:
linksMinutosSalud = []
obtenLinksMinutos(ContenidoMinutosSalud, linksMinutosSalud)

64

#### Tecnologia

In [15]:
linksMinutosTecnologia = []
obtenLinksMinutos(ContenidoMinutosTecnologia, linksMinutosTecnologia)

46

#### Ciencia

In [16]:
linksMinutosCiencia = []
obtenLinksMinutos(ContenidoMinutosCiencia, linksMinutosCiencia)

43

# 2.) Vamos a sacar el contenido que queremos de cada pagina

### Contenido de las noticias de "El Mundo"

In [17]:
# Obtenemos todo el contenido de la noticia
def contenidoArticulolosMundo(array, area): # array con las urls, area (Salud, Tecnología, Ciencia)
    for i in range(20):
        req = requests.get(array[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            archivo = open("ElMundo/"+area+"/"+str(area)+"."+datetime.today().strftime('%Y-%m-%d')+"."+str(i).zfill(3)+".txt", "w")
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="ue-c-article__headline").string
            except:
                titulo = ""
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                subtitulo = soup.find('p', class_="ue-c-article__standfirst").get_text()
            except:
                subtitulo = ""

            archivo.write(str(titulo)+"\n\n"+str(subtitulo)+"\n")

            #Este div contiene todos los "p" con el contenido de la noticia  
            div = soup.findAll('div', class_="ue-l-article__body ue-c-article__body")
            for p in div:
                for each in p.findAll("p"): 
                    archivo.write("\n"+each.get_text())
            archivo.close()
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1

In [ ]:
contenidoArticulolosMundo(linksSalud, "Salud")

In [ ]:
contenidoArticulolosMundo(linksTecnologia, "Tecnologia")

In [ ]:
contenidoArticulolosMundo(linksCiencia, "Ciencia")

### Contenido de las noticias de "El País"

In [ ]:
# Obtenemos todo el contenido de la noticia
def contenidoArticulolosPais(array, area): # array con las urls, area (Salud, Tecnología, Ciencia)
    for i in range(20):
        req = requests.get(array[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            archivo = open("ElPais/"+area+"/"+str(area)+datetime.today().strftime('%Y-%m-%d')+"."+str(i).zfill(3)+".txt", "w")
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="a_t").string
            except:
                titulo = ""
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                subtitulo = soup.find('h2', class_="a_st").get_text()
            except:
                subtitulo = ""

            archivo.write(str(titulo)+"\n\n"+str(subtitulo)+"\n")
            
            # Contenido de la noticia
            div = soup.findAll('div', class_="a_c clearfix")
            for p in div:
                for each in p.findAll("p"): 
                    try:
                        archivo.write("\n"+each.get_text())
                    except:
                        archivo.write(" ")

            archivo.close()
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1

In [ ]:
contenidoArticulolosPais(linksSanidadElPáis, "Salud")

In [ ]:
contenidoArticulolosPais(linksCienciaElPáis, "Ciencia")

In [ ]:
contenidoArticulolosPais(linksTecnologiaElPáis, "Tecnologia")

### Contenido de las noticias de "20 Minutos"

In [ ]:
# Obtenemos todo el contenido de la noticia
def contenidoArticulolosMinutos(array, area): # array con las urls, area (Salud, Tecnología, Ciencia)
    for i in range(20):
        req = requests.get(array[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            archivo = open("20Minutos/"+area+"/"+str(area)+"."+datetime.today().strftime('%Y-%m-%d')+"."+str(i).zfill(3)+".txt", "w")
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="article-title").string
            except:
                titulo = ""
                
            archivo.write(str(titulo)+"\n\n")
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                div = soup.findAll('div', class_="article-intro")
                for li in div:
                    for each in li.findAll("li"): 
                        archivo.write(str(each.get_text()))
            except:
                subtitulo = ""
                
            archivo.write("\n\n")
                
            # Contenido de la noticia
            p = soup.findAll('p', class_="paragraph")
            for cont in p: 
                try:
                    archivo.write(cont.get_text())
                except:
                    archivo.write(" ")
            
            archivo.close()
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1

In [ ]:
contenidoArticulolosMinutos(linksMinutosSalud, "Salud")

In [ ]:
contenidoArticulolosMinutos(linksMinutosCiencia, "Ciencia")

In [ ]:
contenidoArticulolosMinutos(linksMinutosTecnologia, "Tecnologia")

# Métodos para obtener las etiquetas

Este método nos devuelve un dataframe con una columna con el texto de la noticia y otra con las etiquetas de la noticia de El Mundo

He decidido eliminar todos los espacios y caracteres especiales para que las coincidencias entre los diferentes medios sean más faciles

In [18]:
def contenidoEtiquetasElMundo(arraySalud, arrayCiencia, arrayTecnologia, df): # array con las urls, area (Salud, Tecnología, Ciencia)
#------------SALUD-------------
    for i in range(22):
        req = requests.get(arraySalud[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="ue-c-article__headline").string
            except:
                titulo = ""
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                subtitulo = soup.find('p', class_="ue-c-article__standfirst").get_text()
            except:
                subtitulo = ""

            texto = titulo+"\n\n"+subtitulo+"\n"
            #Este div contiene todos los "p" con el contenido de la noticia  
            div = soup.findAll('div', class_="ue-l-article__body ue-c-article__body")
            for p in div:
                for each in p.findAll("p"): 
                    texto = texto+"\n"+str(each.get_text())
            #print(texto)
                    
            #try catch para guardar las etiquetas y los textos en el dataframe
            try:
                li = soup.findAll('li', class_="ue-c-article__tags-item")
                etiquetas = []
                if (li!=[]):
                    for each in li:
                        etiquetas.append(each.get_text())
                        for i in range(len(etiquetas)):
                            etiquetas[i] = re.sub("\ ","",etiquetas[i])
                            etiquetas[i] = etiquetas[i].lower()
                else:
                    etiquetas = ""
            except:
                etiquetas_columna = ""
            
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'data':texto,'etiquetas': etiquetas}
            df=df.append(new_row, ignore_index=True)
            
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1

#------------CIENCIA-------------
    for i in range(22):
        req = requests.get(arrayCiencia[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="ue-c-article__headline").string
            except:
                titulo = ""
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                subtitulo = soup.find('p', class_="ue-c-article__standfirst").get_text()
            except:
                subtitulo = ""

            texto = titulo+"\n\n"+subtitulo+"\n"
            #Este div contiene todos los "p" con el contenido de la noticia  
            div = soup.findAll('div', class_="ue-l-article__body ue-c-article__body")
            for p in div:
                for each in p.findAll("p"): 
                    texto = texto+"\n"+str(each.get_text())
            #print(texto)
                    
            #try catch para guardar las etiquetas y los textos en el dataframe
            try:
                li = soup.findAll('li', class_="ue-c-article__tags-item")
                etiquetas = []
                if (li!=[]):
                    for each in li:
                        etiquetas.append(each.get_text())
                        for i in range(len(etiquetas)):
                            etiquetas[i] = re.sub("\ ","",etiquetas[i])
                            etiquetas[i] = etiquetas[i].lower()
                else:
                    etiquetas = ""
            except:
                etiquetas_columna = ""
            
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'data':texto,'etiquetas': etiquetas}
            df=df.append(new_row, ignore_index=True)
            
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1

#------------TECNOLOGIA-------------
    for i in range(22):
        req = requests.get(arrayTecnologia[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="ue-c-article__headline").string
            except:
                titulo = ""
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                subtitulo = soup.find('p', class_="ue-c-article__standfirst").get_text()
            except:
                subtitulo = ""

            texto = titulo+"\n\n"+subtitulo+"\n"
            #Este div contiene todos los "p" con el contenido de la noticia  
            div = soup.findAll('div', class_="ue-l-article__body ue-c-article__body")
            for p in div:
                for each in p.findAll("p"): 
                    texto = texto+"\n"+str(each.get_text())                    
            #try catch para guardar las etiquetas y los textos en el dataframe
            try:
                li = soup.findAll('li', class_="ue-c-article__tags-item")
                etiquetas = []
                if (li!=[]):
                    for each in li:
                        etiquetas.append(each.get_text())
                        for i in range(len(etiquetas)):
                            etiquetas[i] = re.sub("\ ","",etiquetas[i])
                            etiquetas[i] = etiquetas[i].lower()
                else:
                    etiquetas = ""
            except:
                etiquetas_columna = ""
            
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'data':texto,'etiquetas': etiquetas}
            df=df.append(new_row, ignore_index=True)
            
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1
    df.drop(df.loc[(df['etiquetas']=='')].index, inplace=True)
    #df.reset_index(inplace=True, drop=False)
    return df

Este método nos devuelve un dataframe con una columna con el texto de la noticia y otra con las etiquetas de la noticia de El País

In [19]:
# Obtenemos todo el contenido de la noticia
def contenidoEtiquetasElPais(arraySalud, arrayCiencia, arrayTecnologia, df): # array con las urls, area (Salud, Tecnología, Ciencia)
#------------SALUD-------------
    for i in range(20):
        req = requests.get(arraySalud[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="a_t").string
            except:
                titulo = ""
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                subtitulo = soup.find('h2', class_="a_st").get_text()
            except:
                subtitulo = ""

            texto = titulo+"\n\n"+subtitulo+"\n"
            
            # Contenido de la noticia
            div = soup.findAll('div', class_="a_c clearfix")
            for p in div:
                for each in p.findAll("p"): 
                    texto = texto+"\n"+str(each.get_text())
                    
            # Etiqetas de la noticia
            import re
            ul = soup.find('ul', class_="_df _ls")
            etiquetas = []
            for each in ul:
                etiquetas.append(each.get_text())
                for i in range(len(etiquetas)):
                    etiquetas[i] = re.sub("\ |\\n|\-","",etiquetas[i])
                    etiquetas[i] = etiquetas[i].lower()
            
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'data':texto,'etiquetas': etiquetas}
            df=df.append(new_row, ignore_index=True)
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1
#------------CIENCIA-------------
    for i in range(20):
        req = requests.get(arrayCiencia[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="a_t").string
            except:
                titulo = ""
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                subtitulo = soup.find('h2', class_="a_st").get_text()
            except:
                subtitulo = ""

            texto = titulo+"\n\n"+subtitulo+"\n"
            
            # Contenido de la noticia
            div = soup.findAll('div', class_="a_c clearfix")
            for p in div:
                for each in p.findAll("p"): 
                    texto = texto+"\n"+str(each.get_text())
                    
            # Etiqetas de la noticia
            import re
            ul = soup.find('ul', class_="_df _ls")
            etiquetas = []
            for each in ul:
                etiquetas.append(each.get_text())
                for i in range(len(etiquetas)):
                    etiquetas[i] = re.sub("\ |\\n|\-","",etiquetas[i])
                    etiquetas[i] = etiquetas[i].lower()
            
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'data':texto,'etiquetas': etiquetas}
            df=df.append(new_row, ignore_index=True)
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1

#------------TECNOLOGIA-------------
    for i in range(20):
        req = requests.get(arrayTecnologia[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="a_t").string
            except:
                titulo = ""
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                subtitulo = soup.find('h2', class_="a_st").get_text()
            except:
                subtitulo = ""

            texto = titulo+"\n\n"+subtitulo+"\n"
            
            # Contenido de la noticia
            div = soup.findAll('div', class_="a_c clearfix")
            for p in div:
                for each in p.findAll("p"): 
                    texto = texto+"\n"+str(each.get_text())
                    
            # Etiqetas de la noticia
            try:
                import re
                ul = soup.find('ul', class_="_df _ls")
                etiquetas = []
                for each in ul:
                    etiquetas.append(each.get_text())
                    for i in range(len(etiquetas)):
                        etiquetas[i] = re.sub("\ |\\n|\-","",etiquetas[i])
                        etiquetas[i] = etiquetas[i].lower()
            except:
                etiquetas = ''
            
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'data':texto,'etiquetas': etiquetas}
            df=df.append(new_row, ignore_index=True)
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1


    df.drop(df.loc[df['etiquetas']==''].index, inplace=True)
    #df.reset_index(inplace=True, drop=False)
    return df

Este método nos devuelve un dataframe con una columna con el texto de la noticia y otra con las etiquetas de la noticia de El País

In [20]:
# Obtenemos todo el contenido de la noticia
def contenidoEtiquetasMinutos(arraySalud, arrayCiencia, arrayTecnologia, df): # array con las urls, area (Salud, Tecnología, Ciencia)
#------------SALUD-------------
    for i in range(20):
        req = requests.get(arraySalud[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="article-title").string
            except:
                titulo = ""
                
            texto = str(titulo)+"\n\n"
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                div = soup.findAll('div', class_="article-intro")
                for li in div:
                    for each in li.findAll("li"): 
                        texto = str(texto)+str(each.get_text())
            except:
                subtitulo = ""
                
            texto = texto+"\n"
                
            # Contenido de la noticia
            p = soup.findAll('p', class_="paragraph")
            for cont in p: 
                try:
                    texto = texto+cont.get_text()
                except:
                    texto = str(texto)+""
            
            #sacamos las etiquetas
            li = soup.find_all('li', class_="tag")
            etiquetas = []
            for each in li:
                etiquetas.append(each.get_text())
                for i in range(len(etiquetas)):
                    etiquetas[i] = re.sub("\ |\\n|\-","",etiquetas[i])
                    etiquetas[i] = etiquetas[i].lower()
                
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'data':texto,'etiquetas': etiquetas}
            df=df.append(new_row, ignore_index=True)
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1
        
#------------CIENCIA-------------
    for i in range(20):
        req = requests.get(arrayCiencia[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="article-title").string
            except:
                titulo = ""
                
            texto = str(titulo)+"\n\n"
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                div = soup.findAll('div', class_="article-intro")
                for li in div:
                    for each in li.findAll("li"): 
                        texto = str(texto)+str(each.get_text())
            except:
                subtitulo = ""
                
            texto = texto+"\n"
                
            # Contenido de la noticia
            p = soup.findAll('p', class_="paragraph")
            for cont in p: 
                try:
                    texto = str(texto)+cont.get_text()
                except:
                    texto = str(texto)+""
            
            #sacamos las etiquetas
            li = soup.find_all('li', class_="tag")
            etiquetas = []
            for each in li:
                etiquetas.append(each.get_text())
                for i in range(len(etiquetas)):
                    etiquetas[i] = re.sub("\ |\\n|\-","",etiquetas[i])
                    etiquetas[i] = etiquetas[i].lower()
                
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'data':texto,'etiquetas': etiquetas}
            df=df.append(new_row, ignore_index=True)
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1

#------------TECNOLOGIA-------------
    for i in range(20):
        req = requests.get(arrayTecnologia[i])
        if((req.status_code) == 200):
            soup = BeautifulSoup (req.content, 'html.parser')
            # Con un try catch hacemos que si el articulo no tiene un titulo lo deje en blanco
            try:
                titulo = soup.find('h1', class_="article-title").string
            except:
                titulo = ""
                
            texto = str(titulo)+"\n\n"
                
            # Con un try catch hacemos que si el articulo no tiene un subtitulo lo deje en blanco
            try:
                div = soup.findAll('div', class_="article-intro")
                for li in div:
                    for each in li.findAll("li"): 
                        texto = str(texto)+str(each.get_text())
            except:
                subtitulo = ""
                
            texto = str(texto)+"\n"
                
            # Contenido de la noticia
            p = soup.findAll('p', class_="paragraph")
            for cont in p: 
                try:
                    texto = str(texto)+cont.get_text()
                except:
                    texto = texto+""
            
            #sacamos las etiquetas
            li = soup.find_all('li', class_="tag")
            etiquetas = []
            for each in li:
                etiquetas.append(each.get_text())
                for i in range(len(etiquetas)):
                    etiquetas[i] = re.sub("\ |\\n|\-","",etiquetas[i])
                    etiquetas[i] = etiquetas[i].lower()
                
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'data':texto,'etiquetas': etiquetas}
            df=df.append(new_row, ignore_index=True)
        else:
            print("Error al acceder a las paginas.")
            break
        i=i+1

    df.drop(df.loc[df['etiquetas']==''].index, inplace=True)
    #df.reset_index(inplace=True, drop=False)
    return df

# 3.) Tratamiento de las noticias

Este método guardamos en un dataframe "df" los diferentes ficheros de cada tematica

In [21]:
def cargar_noticias(ruta_20MinSalud,ruta_20MinCiencia,ruta_20MinTecnologia,ruta_MundoCiencia,ruta_MundoSalud,ruta_MundoTecnologia,ruta_PaisCiencia,ruta_PaisSalud,ruta_PaisTecnologia):
    #extraemos todos los ficheros que hay en acda carpeta
    files_20MinSalud = os.listdir(ruta_20MinSalud)
    files_20MinCiencia = os.listdir(ruta_20MinCiencia)
    files_20MinTecnologia = os.listdir(ruta_20MinTecnologia)
    files_MundoCiencia = os.listdir(ruta_MundoCiencia)
    files_MundoSalud = os.listdir(ruta_MundoSalud)
    files_MundoTecnologia = os.listdir(ruta_MundoTecnologia)
    files_PaisCiencia = os.listdir(ruta_PaisCiencia)
    files_PaisSalud = os.listdir(ruta_PaisSalud)
    files_PaisTecnologia = os.listdir(ruta_PaisTecnologia)
    
    #importamos a un df las noticias
    df = pd.DataFrame(columns = ['file','data'])
    
    #Importamos ficheros de 20MinSalud
    for file in files_20MinSalud:
        try:
            fichero_abierto = open(ruta_20MinSalud+"/"+file, "r")
            texto=fichero_abierto.read()
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de 20MinCiencia
    for file in files_20MinCiencia:
        try:
            fichero_abierto = open(ruta_20MinCiencia+"/"+file, "r")
            texto=fichero_abierto.read()
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de 20MinTecnologia
    for file in files_20MinTecnologia:
        try:
            fichero_abierto = open(ruta_20MinTecnologia+"/"+file, "r")
            texto=fichero_abierto.read()
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de MundoCiencia
    for file in files_MundoCiencia:
        try:
            fichero_abierto = open(ruta_MundoCiencia+"/"+file, "r")
            texto=fichero_abierto.read()
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de MundoSalud
    for file in files_MundoSalud:
        try:
            fichero_abierto = open(ruta_MundoSalud+"/"+file, "r")
            texto=fichero_abierto.read()
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de MundoTecnologia
    for file in files_MundoTecnologia:
        try:
            fichero_abierto = open(ruta_MundoTecnologia+"/"+file, "r")
            texto=fichero_abierto.read()
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de PaisCiencia
    for file in files_PaisCiencia:
        try:
            fichero_abierto = open(ruta_PaisCiencia+"/"+file, "r")
            texto=fichero_abierto.read()
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de PaisSanidad
    for file in files_PaisSalud:
        try:
            fichero_abierto = open(ruta_PaisSalud+"/"+file, "r")
            texto=fichero_abierto.read()
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de PaisTecnologia
    for file in files_PaisTecnologia:
        try:
            fichero_abierto = open(ruta_PaisTecnologia+"/"+file, "r")
            texto=fichero_abierto.read()
            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
    return df

In [22]:
def cargar_noticias_20Mins(ruta_20MinSalud,ruta_20MinCiencia,ruta_20MinTecnologia):
    #extraemos todos los ficheros que hay en acda carpeta
    files_20MinSalud = os.listdir(ruta_20MinSalud)
    files_20MinCiencia = os.listdir(ruta_20MinCiencia)
    files_20MinTecnologia = os.listdir(ruta_20MinTecnologia)

    #importamos a un df las noticias
    df = pd.DataFrame(columns = ['file','data'])
    
    #Importamos ficheros de 20MinSalud
    for file in files_20MinSalud:
        try:
            fichero_abierto = open(ruta_20MinSalud+"/"+file, "r")
            texto=fichero_abierto.read()

            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de 20MinCiencia
    for file in files_20MinCiencia:
        try:
            fichero_abierto = open(ruta_20MinCiencia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de 20MinTecnologia
    for file in files_20MinTecnologia:
        try:
            fichero_abierto = open(ruta_20MinTecnologia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
    
    return df

In [23]:
def cargar_noticias_ElMundo(ruta_MundoCiencia,ruta_MundoSalud,ruta_MundoTecnologia):
    files_MundoCiencia = os.listdir(ruta_MundoCiencia)
    files_MundoSalud = os.listdir(ruta_MundoSalud)
    files_MundoTecnologia = os.listdir(ruta_MundoTecnologia)

    df = pd.DataFrame(columns = ['file','data'])

    #Importamos ficheros de MundoCiencia
    for file in files_MundoCiencia:
        try:
            fichero_abierto = open(ruta_MundoCiencia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de MundoSalud
    for file in files_MundoSalud:
        try:
            fichero_abierto = open(ruta_MundoSalud+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de MundoTecnologia
    for file in files_MundoTecnologia:
        try:
            fichero_abierto = open(ruta_MundoTecnologia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
    return df

In [24]:
def cargar_noticias_ElPais(ruta_PaisCiencia,ruta_PaisSalud,ruta_PaisTecnologia):
    files_PaisCiencia = os.listdir(ruta_PaisCiencia)
    files_PaisSalud = os.listdir(ruta_PaisSalud)
    files_PaisTecnologia = os.listdir(ruta_PaisTecnologia)
    
    df = pd.DataFrame(columns = ['file','data'])
    
    #Importamos ficheros de PaisCiencia
    for file in files_PaisCiencia:
        try:
            fichero_abierto = open(ruta_PaisCiencia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de PaisSanidad
    for file in files_PaisSalud:
        try:
            fichero_abierto = open(ruta_PaisSalud+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de PaisTecnologia
    for file in files_PaisTecnologia:
        try:
            fichero_abierto = open(ruta_PaisTecnologia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
    return df

Este metodo compara una noticia con el resto dentro del df y devuelve un dataframe con cada texto y la similitud que tiene con el texto introducido

In [25]:
def compararNoticiasActualzado(textoin, df):
    newdf = pd.DataFrame(columns = ['porcentage','data'])
    
    f = open("stopwords.txt")
    mis_stopwords = f.read() 
    vectorizer = TfidfVectorizer(stop_words=set(mis_stopwords))

    #vectorizer = TfidfVectorizer()
    for i in range(len(df)):
        X = vectorizer.fit_transform([textoin,df.data[i]])
        similarity_matrix = cosine_similarity(X,X)
        porcentage = float(similarity_matrix[1][0])
        #print(porcentaje)
        texto = df.data[i]
        new_row = {'porcentage':porcentage,'data': texto}
        newdf = newdf.append(new_row, ignore_index=True)
        i+=1
    
    newdf = newdf.sort_values('porcentage',ascending=False)
    newdf = newdf.reset_index()
    newdf = newdf.drop(['index'], axis=1)
    newdf = newdf.drop([0],axis=0)
    
    return newdf

Este método recibe una lista de etiquetas y aplica sorensen dice para ver su similitud con el resto de etiquetas

In [26]:
def aplicarSorensenDice(etiqueta_seleccionada, df):
    df['similitud'] = '' #creamos la nueva columna con las similitudes
    A = len(etiqueta_seleccionada)
    a = etiqueta_seleccionada
    for i in range(len(df.etiquetas)):
        b = df.etiquetas[i]
        B = len(df.etiquetas[i])
        AyB = len(set(a).intersection(b))
        sorensen=(2*AyB)/(A+B)
        df.similitud[i] = sorensen
    df = df.sort_values('similitud',ascending=False)
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    return df

# 4.) Ventana (GUI)

In [27]:
# SOLO PARA PRUEBAS
df = pd.DataFrame(columns = ['data','etiquetas'])
df = contenidoEtiquetasElMundo(linksSalud, linksCiencia, linksTecnologia, df)
df = contenidoEtiquetasElPais(linksSanidadElPáis, linksCienciaElPáis, linksTecnologiaElPáis, df)
df = contenidoEtiquetasMinutos(linksMinutosSalud, linksMinutosCiencia, linksMinutosTecnologia, df)

In [28]:
df

,data,etiquetas
0,"Covid hoy, última hora en directo | España sum...","[cienciaysalud, coronavirus, covid19, variante..."
1,Trasplantan riñones de cerdo editados genética...,[cienciaysalud]
2,Una familia de Mérida denuncia que han vacunad...,"[badajoz, vacunas, colegios, covid19, coronavi..."
3,"Síntomas de ómicron: Dolor muscular, fatiga y ...","[coronavirus, covid19, cienciaysalud, variante..."
4,Qué quiere decir realmente convivir con un vir...,"[hbpr, coronavirus, covid19]"
...,...,...
170,Alertan de falta de seguridad en la aplicación...,"[juegosolímpicos, pekín, ciberseguridad, aplic..."
171,Desinstala esta app de tu iPhone: te está cobr...,"[iphone, apple, ios, aplicacionesmóvil, redesy..."
172,Un error del navegador Safari filtra el histor...,"[google, apple, seguridadinformática, internet..."
173,"Rusia desmantela a REvil, la banda de cibercri...","[ciberseguridad, ransomware, ciberataque]"


In [31]:
# MAIN CON LA VENTANA
sg.theme('BlueMono')   # Add a little color to your windows
# All the stuff inside your window. This is the PSG magic code compactor...

listaNumero = ["1", "2", "3", "4", "5"] #numero de noticias que se quieren en el ranking
list1 = ["20 minutos", "El Pais", "El Mundo", "Todos"] #Filtrar las noticias por el medio de origen o seleccionar todos
params = 10
medios = ["20 minutos", "El Pais", "El Mundo"]
categorias = ["Ciencia", "Salud", "Tecnologia"]
noticias = ["noticia 1", "noticia 2", "noticia 3", "noticia 4", "noticia 5", "noticia 6", "noticia 7"]
noticiasPorcentajes = [""]


busqueda = [[sg.Text('Consulta:'), sg.InputText(size=80, key= 'textbox_buscar_noticia')],
            [sg.Text('TOP-N:'),sg.Combo(listaNumero, size = params, key= 'combo_topN_consulta'),sg.Text('             '), sg.Text('Filtrar:'), sg.Combo(list1, size = params, key = 'combo_filtrar'),sg.Text('                              '), sg.Button('Buscar', size=10, key='btn_buscarNoticias')],
            [sg.Text('')],
            [sg.Text('Ranking:'), sg.Button('Preview', key='btn_preview_busqueda'), sg.Text('                                                      '), sg.Text('Texto de la noticia:')],
            [sg.Listbox(values=[], size=(30, 10), key='listbox_ranking_noticias'),sg.Text('             ') ,sg.Multiline(size = (50, 10), key = 'textbox_preview_noticia')],
            [sg.Text('')],
            [sg.Button('Salir', size=10)]
         ]
comparar = [[sg.Text('Medio:'), sg.Text('                                  '), sg.Text('Categoría:'), sg.Text('                                                             '), sg.Text('Noticias:')],
            [sg.Listbox(medios, size=(20, 4), key='listbox_medios'),sg.Text('     '), sg.Listbox(categorias, size=(20, 4), key='listbox_categorias'),sg.Text('     '), sg.Button("Ver\nnoticias", size=(10,2), key='btn_verNoticias'),sg.Text('     '), sg.Listbox(values=[], size=(30, 4), key='listbox_noticias')],
            [sg.Button('Preview', key='btn_preview')],
            [sg.Multiline(size = (107, 7), key = 'textbox_preview')],
            [sg.Text('                                      '), sg.Text('TOP-N:'), sg.Combo(listaNumero, size = params, key='combo_topN'),sg.Text('             '), sg.Button('Buscar', size=10, key='btn_buscar')],
            [sg.Text('')],
            [sg.Text('                        '), sg.Text('Ranking:'), sg.Listbox(values="", size=(30, 5), key='noticias_porcentajes'), sg.Button('Preview', key='btn_preview_reslutado')],
            [sg.Text('Texto de la noticia:')],
            [sg.Multiline(size = (107, 7), key = 'textbox_preview_reslutado')],
            [sg.Button('Salir', size=10)]
         ]

recomendar = [[sg.Text('Etiquetas:'), sg.Button('Preview', key='btn_preview_etiquetas_recomendar')],
            [sg.Text('              '), sg.Listbox(values="", size=(90, 7), key='listbox_etiquetas_recomendar')],
            [sg.Text('                                      '), sg.Text('TOP-N:'), sg.Combo(listaNumero, size = params, key='combo_topN_recomendar'), sg.Text('             '), sg.Button('Buscar', size=10, key='btn_buscar_recomendar')],
            [sg.Text('')],
            [sg.Text('                        '), sg.Text('Ranking:'), sg.Listbox(values="", size=(50, 5), key='noticias_porcentajes_recomendar'), sg.Button('Preview', key='btn_preview_reslutado_recomendar')],
            [sg.Text('Texto de la noticia:')],
            [sg.Multiline(size = (107, 13), key = 'textbox_preview_reslutado_recomendar')],
            [sg.Button('Salir', size=10)]
         ]

tabgrp = [
    [sg.TabGroup([[ sg.Tab('Comparador de noticias', comparar),
                    sg.Tab('Buscador de noticias', busqueda),
                    sg.Tab('Recomendador de noticias', recomendar)
                ]], tab_location='top')]]

# Create the Window
window = sg.Window('Buscador y comparador de noticias', tabgrp)
# Event Loop to process "events"
while True:             
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'Salir'):
        break

    if event == "btn_verNoticias":
        eleccion = values['listbox_medios']+values['listbox_categorias']
        print(eleccion)
        if(eleccion==['20 minutos', 'Ciencia']):
            noticias = os.listdir("20Minutos/Ciencia")
            window.FindElement('listbox_noticias').Update(values=noticias)
            print(values['listbox_noticias'])
        elif(eleccion==['20 minutos', 'Salud']):
            noticias = os.listdir("20Minutos/Salud")
            window.FindElement('listbox_noticias').Update(values=noticias)
            print(values['listbox_noticias'])
        elif(eleccion==['20 minutos', 'Tecnologia']):
            noticias = os.listdir("20Minutos/Tecnologia")
            window.FindElement('listbox_noticias').Update(values=noticias)
            print(values['listbox_noticias'])
            
        elif(eleccion==['El Pais', 'Ciencia']):
            noticias = os.listdir("ElPais/Ciencia")
            window.FindElement('listbox_noticias').Update(values=noticias)
            print(values['listbox_noticias'])
        elif(eleccion==['El Pais', 'Salud']):
            noticias = os.listdir("ElPais/Salud")
            window.FindElement('listbox_noticias').Update(values=noticias)
            print(values['listbox_noticias'])
        elif(eleccion==['El Pais', 'Tecnologia']):
            noticias = os.listdir("ElPais/Tecnologia")
            window.FindElement('listbox_noticias').Update(values=noticias)
            print(values['listbox_noticias'])
            
        elif(eleccion==['El Mundo', 'Ciencia']):
            noticias = os.listdir("ElMundo/Ciencia")
            window.FindElement('listbox_noticias').Update(values=noticias)
            print(values['listbox_noticias'])
        elif(eleccion==['El Mundo', 'Salud']):
            noticias = os.listdir("ElMundo/Salud")
            window.FindElement('listbox_noticias').Update(values=noticias)
            print(values['listbox_noticias'])
        elif(eleccion==['El Mundo', 'Tecnologia']):
            noticias = os.listdir("ElMundo/Tecnologia")
            window.FindElement('listbox_noticias').Update(values=noticias)
            print(values['listbox_noticias'])
            
    if event=="btn_preview":
        noticia = values['listbox_noticias']
        f = open(eleccion[0].replace(" ", "")+"/"+eleccion[1]+"/"+noticia[0], "r")
        texto = f.read()
        window['textbox_preview'].update(texto)
        
    if event=="btn_buscar":
        eleccion = values['listbox_medios']+values['listbox_categorias']
        noticia = values['listbox_noticias']
        topN=values['combo_topN']
        dirNoticia = open(eleccion[0].replace(" ", "")+"/"+eleccion[1]+"/"+noticia[0], "r")
        textoNoticia = dirNoticia.read()
        df = cargar_noticias("20Minutos/Salud","20Minutos/Ciencia","20Minutos/Tecnologia","ElMundo/Ciencia","ElMundo/Salud","ElMundo/Tecnologia","ElPais/Ciencia","ElPais/Salud","ElPais/Tecnologia")
        dataRanking = compararNoticiasActualzado(textoNoticia,df)
        rankingLimitado = []
        textoRanking = []
        for i in range(int(topN)):
            i+=1
            # REVISAR
            rankingLimitado.append(str(int(dataRanking.porcentage[i]*100))+" % de coincidencia")
            textoRanking.append(dataRanking.data[i])   

        window.find_element('noticias_porcentajes').Update(values=rankingLimitado)
        
    if event=="btn_preview_reslutado":
        textoResultado = ""        
        for i in range(len(rankingLimitado)):
            if str(values['noticias_porcentajes'][0])==str(rankingLimitado[i]):   
                textoResultado = textoRanking[i]
            i+=1
        window['textbox_preview_reslutado'].update(textoResultado)
    
        #window['textbox_preview_noticia'].update(textoNoticia)
        #window['textbox_preview_noticia'].update(textoNoticia)
    
    if event=="btn_preview_busqueda":
        textoResultado = ""        
        for i in range(len(rankingLimitado2)):
            if str(values['listbox_ranking_noticias'][0])==str(rankingLimitado2[i]):   
                textoResultado = textoRanking2[i]
            i+=1
        window['textbox_preview_noticia'].update(textoResultado)
        
    if event == "btn_buscarNoticias":
        consulta = values['textbox_buscar_noticia']
        topN=values['combo_topN_consulta']
        filtro = values['combo_filtrar']
        print(filtro)
        with open('consulta.txt', 'w') as f:
            f.write(consulta)
        dirNoticia2 = open("consulta.txt", "r")
        textoConsulta = dirNoticia2.read()
        #df = cargar_noticias("20Minutos/Salud","20Minutos/Ciencia","20Minutos/Tecnologia","ElMundo/Ciencia","ElMundo/Salud","ElMundo/Tecnologia","ElPais/Ciencia","ElPais/Salud","ElPais/Tecnologia")
        if(filtro=='20 minutos'):
            df = cargar_noticias_20Mins("20Minutos/Salud","20Minutos/Ciencia","20Minutos/Tecnologia")
            dataRanking2 = compararNoticiasActualzado(textoConsulta,df)
            rankingLimitado2 = []
            textoRanking2 = []
            for i in range(int(topN)):
                i+=1
                # REVISAR
                rankingLimitado2.append(str(int(dataRanking2.porcentage[i]*100))+" % de coincidencia")
                textoRanking2.append(dataRanking2.data[i])   

            window.find_element('listbox_ranking_noticias').Update(values=rankingLimitado2)

        elif(filtro=='El Pais'):
            df = cargar_noticias_ElPais("ElPais/Ciencia","ElPais/Salud","ElPais/Tecnologia")
            dataRanking2 = compararNoticiasActualzado(textoConsulta,df)
            rankingLimitado2 = []
            textoRanking2 = []
            for i in range(int(topN)):
                i+=1
                # REVISAR
                rankingLimitado2.append(str(int(dataRanking2.porcentage[i]*100))+" % de coincidencia")
                textoRanking2.append(dataRanking2.data[i])   

            window.find_element('listbox_ranking_noticias').Update(values=rankingLimitado2)

        elif(filtro=='El Mundo'):
            df = cargar_noticias_ElMundo("ElMundo/Ciencia","ElMundo/Salud","ElMundo/Tecnologia")
            dataRanking2 = compararNoticiasActualzado(textoConsulta,df)
            rankingLimitado2 = []
            textoRanking2 = []
            for i in range(int(topN)):
                i+=1
                # REVISAR
                rankingLimitado2.append(str(int(dataRanking2.porcentage[i]*100))+" % de coincidencia")
                textoRanking2.append(dataRanking2.data[i])   

            window.find_element('listbox_ranking_noticias').Update(values=rankingLimitado2)

        elif(filtro=='Todos'):
            df = cargar_noticias("20Minutos/Salud","20Minutos/Ciencia","20Minutos/Tecnologia","ElMundo/Ciencia","ElMundo/Salud","ElMundo/Tecnologia","ElPais/Ciencia","ElPais/Salud","ElPais/Tecnologia")
            dataRanking2 = compararNoticiasActualzado(textoConsulta,df)
            rankingLimitado2 = []
            textoRanking2 = []
            for i in range(int(topN)):
                i+=1
                # REVISAR
                rankingLimitado2.append(str(int(dataRanking2.porcentage[i]*100))+" % de coincidencia")
                textoRanking2.append(dataRanking2.data[i])   

            window.find_element('listbox_ranking_noticias').Update(values=rankingLimitado2)
            
        else:
            df = cargar_noticias("20Minutos/Salud","20Minutos/Ciencia","20Minutos/Tecnologia","ElMundo/Ciencia","ElMundo/Salud","ElMundo/Tecnologia","ElPais/Ciencia","ElPais/Salud","ElPais/Tecnologia")
            dataRanking2 = compararNoticiasActualzado(textoConsulta,df)
            rankingLimitado2 = []
            textoRanking2 = []
            for i in range(int(topN)):
                i+=1
                # REVISAR
                rankingLimitado2.append(str(int(dataRanking2.porcentage[i]*100))+" % de coincidencia")
                textoRanking2.append(dataRanking2.data[i])   

            window.find_element('listbox_ranking_noticias').Update(values=rankingLimitado2)

        #dataRanking2 = compararNoticiasActualzado(textoConsulta,df)
        #rankingLimitado2 = []
        #textoRanking2 = []
        #for i in range(int(topN)):
         #   i+=1
            #REVISAR
          #  rankingLimitado2.append(str(int(dataRanking2.porcentage[i]*100))+" % de coincidencia")
           # textoRanking2.append(dataRanking2.data[i])   

        #window.find_element('listbox_ranking_noticias').Update(values=rankingLimitado2)

    if event == "btn_preview_etiquetas_recomendar":
        #df = pd.DataFrame(columns = ['data','etiquetas'])
        #df = contenidoEtiquetasElMundo(linksSalud, linksCiencia, linksTecnologia, df)
        #df = contenidoEtiquetasElPais(linksSanidadElPáis, linksCienciaElPáis, linksTecnologiaElPáis, df)
        #df = contenidoEtiquetasMinutos(linksMinutosSalud, linksMinutosCiencia, linksMinutosTecnologia, df)
        window.find_element('listbox_etiquetas_recomendar').Update(values=df.etiquetas)
    
    if event == "btn_buscar_recomendar":
        etiqueta_seleccionada = values["listbox_etiquetas_recomendar"]
        topN = values["combo_topN_recomendar"]
        print(etiqueta_seleccionada)
        # Ahora llamamos al método que aplicará sorensen dice entre las etiquetas seleccionadas y las del dataframe
        newdf = aplicarSorensenDice(etiqueta_seleccionada[0], df)
        resultados_etiquetas2 = []
        ranking_texto2 = []
        for i in range(int(topN[0])):
            resultados_etiquetas2.append("Similitud de etiquetas: "+ str(newdf.similitud[i]))
            ranking_texto2.append(str(newdf.data[i]))
        window.find_element('noticias_porcentajes_recomendar').Update(values=resultados_etiquetas2)
        
    if event == "btn_preview_reslutado_recomendar":
        texto_Resultado = ""        
        for i in range(len(ranking_texto2)):
            if str(values['noticias_porcentajes_recomendar'][0])==str(resultados_etiquetas2[i]):   
                textoResultado = ranking_texto2[i]
            i+=1
        window['textbox_preview_reslutado_recomendar'].update(textoResultado)

window.close()

[['cienciaysalud']]
[['iphone', 'realidadvirtual', 'appleinc.']]
[['ciencia', 'universo', 'bigbang', 'estrellas', 'astrofísica']]
[['ciencia', 'tonga', 'erupcionesvolcanes', 'volcanes', 'tsunami', 'polinesia', 'meteorología', 'aemet', 'desastresnaturales', 'españa']]
[['gimnasio', 'nutrición', 'proteínas']]


# PRUEBA.) Tratamiento de las noticias

In [ ]:
# Similitudes de un documento con el resto (el  resto estan en un dataframe)
def compararNoticias(texto, df):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    similitudes = []
    vectorizer = TfidfVectorizer()

    for i in range(len(df)):
        try:
            X = vectorizer.fit_transform([texto,df.data[i]])
            similarity_matrix = cosine_similarity(X,X)
            similitudes.append(float(similarity_matrix[1][0]))
            i+=1
        except:
            i+=1
    similitudes.sort(reverse=True) # Los ordemanos de mayor a menor 
    similitudes.remove(similitudes[0]) # Eliminamos el primer valor porque nos da la similitud con el propio elemento, por loo que siempre va a ser 1 (no nos sirve de nada)
    return similitudes

Este método guardamos en un dataframe "df" los diferentes ficheros de cada tematica

In [ ]:
def cargar_noticias(ruta_20MinSalud,ruta_20MinCiencia,ruta_20MinTecnologia,ruta_MundoCiencia,ruta_MundoSalud,ruta_MundoTecnologia,ruta_PaisCiencia,ruta_PaisSanidad,ruta_PaisTecnologia):
    import os
    #extraemos todos los ficheros que hay en acda carpeta
    files_20MinSalud = os.listdir(ruta_20MinSalud)
    files_20MinCiencia = os.listdir(ruta_20MinCiencia)
    files_20MinTecnologia = os.listdir(ruta_20MinTecnologia)
    files_MundoCiencia = os.listdir(ruta_MundoCiencia)
    files_MundoSalud = os.listdir(ruta_MundoSalud)
    files_MundoTecnologia = os.listdir(ruta_MundoTecnologia)
    files_PaisCiencia = os.listdir(ruta_PaisCiencia)
    files_PaisSanidad = os.listdir(ruta_PaisSanidad)
    files_PaisTecnologia = os.listdir(ruta_PaisTecnologia)
    
    #importamos a un df las noticias
    import pandas as pd
    df = pd.DataFrame(columns = ['file','data'])
    
    #Importamos ficheros de 20MinSalud
    for file in files_20MinSalud:
        try:
            fichero_abierto = open(ruta_20MinSalud+"/"+file, "r")
            texto=fichero_abierto.read()

            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de 20MinCiencia
    for file in files_20MinCiencia:
        try:
            fichero_abierto = open(ruta_20MinCiencia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de 20MinTecnologia
    for file in files_20MinTecnologia:
        try:
            fichero_abierto = open(ruta_20MinTecnologia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de MundoCiencia
    for file in files_MundoCiencia:
        try:
            fichero_abierto = open(ruta_MundoCiencia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de MundoSalud
    for file in files_MundoSalud:
        try:
            fichero_abierto = open(ruta_MundoSalud+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de MundoTecnologia
    for file in files_MundoTecnologia:
        try:
            fichero_abierto = open(ruta_MundoTecnologia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de PaisCiencia
    for file in files_PaisCiencia:
        try:
            fichero_abierto = open(ruta_PaisCiencia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de PaisSanidad
    for file in files_PaisSanidad:
        try:
            fichero_abierto = open(ruta_PaisSanidad+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
            
    #Importamos ficheros de PaisTecnologia
    for file in files_PaisTecnologia:
        try:
            fichero_abierto = open(ruta_PaisTecnologia+"/"+file, "r")
            texto=fichero_abierto.read()
        
            #La lista de parada se ejecuta despues al vectorizar

            #Añadimos el nombre del fichero al df y su tipo
            new_row = {'file':file,'data': texto}
            df=df.append(new_row, ignore_index=True)
        except:
            fichero_abierto.close()
    return df

In [ ]:
df = cargar_noticias("20Minutos/Salud","20Minutos/Ciencia","20Minutos/Tecnologia","ElMundo/Ciencia","ElMundo/Salud","ElMundo/Tecnologia","ElPais/Ciencia","ElPais/Salud","ElPais/Tecnologia")
df

In [ ]:
# Vectorizar un documento (obtenemos los vectores de los documentos que queramos)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer ()

X = vectorizer.fit_transform([df.data[1],df.data[130]])
vectorizer.get_feature_names()
vectorDoc = X.toarray()


In [ ]:
#TF-IDF. Similitud entre dos documentos
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer ()
X = vectorizer.fit_transform([df.data[1],df.data[250]])
similarity_matrix = cosine_similarity(X,X)
similarity_matrix[1][0]

In [ ]:
# Similitudes de un documento con el resto
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
similitudes = []
for i in range(len(df)):
    vectorizer = TfidfVectorizer ()
    try:
        vectorizer = TfidfVectorizer ()
        X = vectorizer.fit_transform([df.data[250],df.data[i]])
        similarity_matrix = cosine_similarity(X,X)
        similitudes.append(float(similarity_matrix[1][0]))
        i+=1
    except:
        i+=1
similitudes.sort(reverse=True) # Los ordemanos de mayor a menor 
similitudes.remove(similitudes[0]) # Eliminamos el primer valor porque nos da la similitud con el propio elemento, por loo que siempre va a ser 1 (no nos sirve de nada)

In [ ]:
print(similitudes)

In [ ]:
# Vectorizar un documento y calcular su similitud con todos los documentos
from sklearn.feature_extraction.text import TfidfVectorizer

for i in range(len(df)):
    vectorizer = TfidfVectorizer ()
    try:
        X = vectorizer.fit_transform([df.data[1],df.data[i]])
        vectorDoc = X.toarray()
        print(vectorDoc)
        i+=1
    except:
        i+=1

In [ ]:
# Se define una lista con los símbolos de puntuación y las stopwords para que sean retiradas del texto
from string import punctuation
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

lista_parada = open(os.getcwd()+"/stopwords.txt","r").read().split()
non_words = list(punctuation)

In [ ]:
# funciones para el preprocesado del texto
def remove_stop_words(dirty_text):
    cleaned_text = ''
    for word in dirty_text.split():
        if word in language_stopwords or word in non_words:
            continue
        else:
            cleaned_text += word + ' '
    return cleaned_text

def remove_punctuation(dirty_string):
    for word in non_words:
        dirty_string = dirty_string.replace(word, '')
    return dirty_string

def process_file(file_name):
    file_content = open(file_name, "r").read()
    # All to lower case
    file_content = file_content.lower()
    # Remove punctuation and spanish stopwords
    file_content = remove_punctuation(file_content)
    file_content = remove_stop_words(file_content)
    return file_content

def procesado(ruta):
    for file in (os.listdir(ruta)):
        process_file(ruta)

In [ ]:
procesado("20Minutos/Salud")

In [ ]:
# Hacemos el preprocesado del texto
import os

os.listdir("20Minutos/Salud")
os.listdir("20Minutos/Ciencia")
os.listdir("20Minutos/Tecnologia")
os.listdir("ElMundo/Ciencia")
os.listdir("ElMundo/Salud")
os.listdir("ElMundo/Tecnologia")
os.listdir("ElPais/Ciencia")
os.listdir("ElPais/Sanidad")
os.listdir("ElPais/Tecnologia")